In [49]:
import openpyxl as xl
import pandas as pd
import html as ht
import requests as rq
from bs4 import BeautifulSoup as bs
import re

In [117]:
#Take urls from csv
from numpy import NaN

df_url_list = pd.read_csv('Speeches_data.csv',delimiter=';')[['country_name','URL_text']]
list_url = list(df_url_list['URL_text'])
Use_url = [i for i in list_url if '.htm' in i]

In [120]:
 #to append all text for analysis

# response = rq.get('https://www.bis.org/review/r160616e.htm')
# soup_1 = bs(response.text,'html.parser')
# #navigating through html tags to get body text
# text_1 = soup_1.find('body').find('div',attrs={'id':'body'}).find('div',attrs={'id':'bispage'}).find('div',attrs={'id':'pagecontent'}).text
# clean_text = (re.sub('\n','',text_1.strip()).split(' '))[719:]



[]


In [170]:
#we will append the text results to a list of values and later asign a key to each list
all_text = []
for i in Use_url:
    print(i)
    response = rq.get(i)
    soup = bs(response.text,'html.parser')
    text = soup.find('body').find('div',attrs={'id':'container'}).find('div',attrs={'id':'cmsContent'}).text
    #text = soup.find('body').find('div',attrs={'id':'body'}).find('div',attrs={'id':'bispage'}).find('div',attrs={'id':'pagecontent'}).find('div',attrs={'class':'fullwidth-inner'}).find('div',attrs={'class':''}).text
    cleaned_text = (re.sub('\n','',text.strip()).split(' '))
    # cleaned_text = cleaned_text[cleaned_text.index('')]
    all_text.append(cleaned_text)
    print('next file')
print(all_text)

https://www.bis.org/review/r160426b.htm
next file
https://www.bis.org/review/r160616e.htm
next file
https://www.bis.org/review/r160621e.htm
next file
https://www.bis.org/review/r160622a.htm
next file
https://www.bis.org/review/r161118a.htm
next file
https://www.bis.org/review/r161128a.htm
next file
https://www.bis.org/review/r161214a.htm
next file
https://www.bis.org/review/r170117b.htm
next file
https://www.bis.org/review/r170309b.htm
next file
https://www.bis.org/review/r170425h.htm
next file
https://www.bis.org/review/r170609c.htm
next file
https://www.bis.org/review/r170621b.htm
next file
https://www.bis.org/review/r170720b.htm
next file
https://www.bis.org/review/r170731e.htm
next file
https://www.bis.org/review/r170904d.htm
next file
https://www.bis.org/review/r170921d.htm
next file
https://www.bis.org/review/r171009f.htm
next file
https://www.bis.org/review/r171010b.htm
next file
https://www.bis.org/review/r171031c.htm
next file
https://www.bis.org/review/r171102h.htm
next file


In [152]:
print(all_text)

[]
